In [3]:
from wildlife_datasets.datasets import AnimalCLEF2025

dataset = AnimalCLEF2025(root)

ModuleNotFoundError: No module named 'wildlife_datasets'

In [2]:
from huggingface_hub import snapshot_download
snapshot_download("facebook/vit-base-patch16-224", cache_dir="./vit_weights")

LocalEntryNotFoundError: An error happened while trying to locate the files on the Hub and we cannot find the appropriate snapshot folder for the specified revision on the local disk. Please check your internet connection and try again.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
import pandas as pd
from PIL import Image
import os
import json
from tqdm import tqdm

# ===== Configuration =====
root_dir = "/kaggle/input/animal-clef-2025"
metadata_path = os.path.join(root_dir, "metadata.csv")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 224
batch_size = 32
num_workers = 2
confidence_threshold = 0.80
num_epochs = 250
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# ===== Load metadata =====
df = pd.read_csv(metadata_path)

# ===== Dataset Class =====
class AnimalReIDDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform
        self.label2id = {label: i for i, label in enumerate(sorted(self.df['identity'].unique()))}
        self.id2label = {v: k for k, v in self.label2id.items()}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(root_dir, row["path"])
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        label = self.label2id[row["identity"]]
        return img, label

# ===== Image Transform =====
transform = transforms.Compose([
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# ===== Build Dataset and Dataloader =====
train_df = df[(df["split"] == "database") & (df["identity"].notna())]
train_dataset = AnimalReIDDataset(train_df, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# ===== Model Definition =====
class MAEClassifier(nn.Module):
    def __init__(self, num_classes, freeze_encoder=True):
        super(MAEClassifier, self).__init__()
        self.encoder = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=0)
        self.encoder_frozen = freeze_encoder

        if self.encoder_frozen:
            for param in self.encoder.parameters():
                param.requires_grad = False

        self.classifier = nn.Linear(self.encoder.num_features, num_classes)

    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

    def unfreeze_encoder(self, num_blocks=6):
        if self.encoder_frozen:
            print(f"Unfreezing last {num_blocks} encoder blocks for fine-tuning...")
            for block in self.encoder.blocks[-num_blocks:]:
                for param in block.parameters():
                    param.requires_grad = True
            for param in self.encoder.norm.parameters():
                param.requires_grad = True
            self.encoder_frozen = False

# ===== Initialize or Resume Model =====
model = MAEClassifier(num_classes=len(train_dataset.label2id), freeze_encoder=True).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

start_epoch = 0
best_loss = float('inf')
checkpoint_path = os.path.join(checkpoint_dir, "last_checkpoint.pth")

if os.path.exists(checkpoint_path):
    print("Resuming from last checkpoint...")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    best_loss = checkpoint["best_loss"]
    model.encoder_frozen = checkpoint.get("encoder_frozen", True)
    print(f"Resumed from epoch {start_epoch} with best loss {best_loss:.4f}")

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs - start_epoch)
scaler = torch.cuda.amp.GradScaler()

# ===== Training Loop =====
for epoch in range(start_epoch, num_epochs):
    if epoch == 10 and model.encoder_frozen:
        model.unfreeze_encoder(num_blocks=6)
        optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs - epoch)

    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs} - Total Loss: {total_loss:.4f}")

    if total_loss < best_loss:
        best_loss = total_loss
        torch.save(model.state_dict(), os.path.join(checkpoint_dir, "best_model.pth"))
        print(" Best model saved.")

    if (epoch + 1) % 20 == 0:
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "best_loss": best_loss,
            "encoder_frozen": model.encoder_frozen
        }, os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pth"))
        print(f"Checkpoint saved at epoch {epoch+1}")

torch.save({
    "epoch": epoch,
    "model_state_dict": model.state_dict(),
    "best_loss": best_loss,
    "encoder_frozen": model.encoder_frozen
}, checkpoint_path)
print("Last checkpoint saved.")

with open(os.path.join(checkpoint_dir, "label_map.json"), "w") as f:
    json.dump(train_dataset.id2label, f)
print(" Label map saved.")





####you can run all the above at once for training then run the one below 









model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/tmp/ipykernel_31/3235166995.py:108: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/250:   0%|          | 0/409 [00:00<?, ?it/s]/tmp/ipykernel_31/3235166995.py:124: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/250 - Total Loss: 2499.9117
 Best model saved.


Epoch 2/250 - Total Loss: 2239.5651
 Best model saved.


Epoch 3/250 - Total Loss: 2156.8118
 Best model saved.


...

Epoch 128/250 - Total Loss: 433.3189


Epoch 129/250 - Total Loss: 432.4508


Epoch 130/250 - Total Loss: 432.2581


Epoch 131/250 - Total Loss: 432.1827


Epoch 132/250:  51%|█████     | 209/409 [01:09<01:01,  3.26it/s, loss=1.06]

In [ ]:
###Inference code

import torch.nn.functional as F 
checkpoint_dir = "./checkpoints"
checkpoint_path = os.path.join(checkpoint_dir, "best_model.pth")
label_map_path = os.path.join(checkpoint_dir, "label_map.json")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 224
confidence_threshold = 0.80

# ===== Load metadata and label map =====
df = pd.read_csv(metadata_path)
with open(label_map_path, "r") as f:
    id2label = json.load(f)
label_map = {int(k): v for k, v in id2label.items()}
num_classes = len(label_map)

# ===== Transform =====
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# ===== Model Definition =====
class MAEClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super(MAEClassifier, self).__init__()
        import timm
        self.encoder = timm.create_model("vit_base_patch16_224", pretrained=False, num_classes=0)
        self.classifier = torch.nn.Linear(self.encoder.num_features, num_classes)

    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

# ===== Load Model from Checkpoint =====
model = MAEClassifier(num_classes=num_classes).to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()
print(f" Loaded checkpoint from {checkpoint_path}")

# ===== Evaluate on Query Set =====
results = []
query_df = df[df["split"] == "query"].reset_index(drop=True)

for _, row in tqdm(query_df.iterrows(), total=len(query_df), desc="Evaluating"):
    img_path = os.path.join(root_dir, row["path"])
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(img_tensor)
        probs = F.softmax(logits, dim=1)
        max_prob, pred_class = torch.max(probs, dim=1)
        identity = label_map[pred_class.item()] if max_prob.item() >= confidence_threshold else "new_individual"

    results.append({
        "image_id": row["image_id"],
        "identity": identity
    })

# ===== Save Submission =====
submission_df = pd.DataFrame(results)
submission_df.to_csv("submission.csv", index=False)
print(" Final classifier-based submission saved as 'submission.csv'")